<a href="https://colab.research.google.com/github/amcheyre-nw/medical_images/blob/main/ece495_mil_ana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Copy data from Google Drive to local storage

The data for the project is hosted on Google Drive. Reading files from Google Drive is slow, so we copy .zip archives containing the .tfr files to local storage to accelerate training and inference.

The .zip archives are > 20 GB so the copy and decompression will take ~10 minutes.

In [1]:
from google.colab import drive
import os
import random

# mount google drive containing data, mil library
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# define paths for data
project_drive = "/content/drive/MyDrive/Medical_Images_Data/project/"
data_drive = project_drive + "data/"
table_drive = data_drive + "dataset_ece495_2022.csv" # dataset description .csv
local = "./" # root local destination for unzipped data in ./train, ./validation
datasets = [("validation", 53), ("train", 417)]

# helper function to filter files of a specific type in a directory
def filter(path, ext=".tfr"):
  return [f'{path}{f}' for f in os.listdir(path) if os.path.splitext(f)[1] == ext]

# copy data to local storage for fast i/o
for (dataset, count) in datasets:
  
  # formulate source .zip filename, destination path, destination filename
  src = data_drive + dataset + "_ece495_2022.zip"
  dest_path = local + dataset + "/"
  dest_file = local + dataset + "_ece495_2022.zip"

  # copy, unzip, and delete .zip
  if not os.path.exists(dest_path):
    !mkdir {dest_path}
  if not os.path.isfile(dest_file):
    !cp {src} {dest_file}
  !unzip {dest_file} -d {dest_path}
  !rm {dest_file}

  # check that the correct number of .tfr files are found
  files = filter(dest_path)
  assert len(files) == count

# create file lists
train = filter('./train/')
validation = filter('./validation/')

# shuffle training files to avoid batching blocks of similar samples
random.Random(1).shuffle(train)

Archive:  ./validation_ece495_2022.zip
  inflating: ./validation/TCGA-02-0058-01Z-00-DX1.EfficientNetV2S_256_0_20X.tfr  
  inflating: ./validation/TCGA-02-0070-01Z-00-DX1.EfficientNetV2S_256_0_20X.tfr  
  inflating: ./validation/TCGA-02-0269-01Z-00-DX1.EfficientNetV2S_256_0_20X.tfr  
  inflating: ./validation/TCGA-02-0289-01Z-00-DX2.EfficientNetV2S_256_0_20X.tfr  
  inflating: ./validation/TCGA-02-0326-01Z-00-DX2.EfficientNetV2S_256_0_20X.tfr  
  inflating: ./validation/TCGA-06-0125-01Z-00-DX2.EfficientNetV2S_256_0_20X.tfr  
  inflating: ./validation/TCGA-06-0129-01Z-00-DX1.EfficientNetV2S_256_0_20X.tfr  
  inflating: ./validation/TCGA-06-0143-01Z-00-DX2.EfficientNetV2S_256_0_20X.tfr  
  inflating: ./validation/TCGA-06-0171-01Z-00-DX2.EfficientNetV2S_256_0_20X.tfr  
  inflating: ./validation/TCGA-06-0173-01Z-00-DX3.EfficientNetV2S_256_0_20X.tfr  
  inflating: ./validation/TCGA-06-0184-01Z-00-DX1.EfficientNetV2S_256_0_20X.tfr  
  inflating: ./validation/TCGA-06-0195-01Z-00-DX2.Efficient

### Install Pandas and MIL packages and import functions

Pandas will be used to interact with the subject table describing the dataset. The MIL library is used to read the .tfr files.



In [3]:
# install MIL package and package dependencies OpenSlide and HistomicsStream
!apt-get install -y openslide-tools
!pip install openslide-python
!pip install histomics_stream 'large_image[openslide]' scikit_image --find-links https://girder.github.io/large_image_wheels
!pip install {project_drive + "mil/"}[ray]

# imports
from mil.metrics import Balanced, F1, Mcc, Sensitivity, Specificity
from mil.models import attention_flat
from mil.io.reader import read_record, peek
import tensorflow as tf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 12 not upgraded.
Need to get 92.5 kB of archives.
After this operation, 268 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopenslide0 amd64 3.4.1+dfsg-2 [79.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 openslide-tools amd64 3.4.1+dfsg-2 [12.7 kB]
Fetched 92.5 kB in 1s (124 kB/s)
Selecting previously unselected package libopenslide0.
(Reading database ... 123934 files and directories currently installed.)
Preparing to unpack .../libopenslide0_3.4.1+dfsg-2

### Examine contents of a .tfr file and flattened versus structured forms
Let's look at what is contained in the .tfr files, and the differences in flattened versus structured formats.

In [4]:
# helper function for printing dictionaries
def print_dict(d):
  for key in d:
    print(f"\t{key}: {d[key]}, {d[key].dtype}")

# inspect a single file to retrieve saved variable names
serialized = list(tf.data.TFRecordDataset(train[0]))[0]
variables = peek(serialized)

# avoid decoding unecessary variables
for v in ['label_gender', 'label_histology', 'label_subtype']:
  variables.pop(v, None)

# read the contents of this same file in flattened format and inspect
features, labels, slide_meta, tile_meta = read_record(serialized, variables, structured=False)
print(f"features (flattened): \n\t{features.shape}, {features.dtype}")
print("labels:")
print_dict(labels)
print("slide_meta:")
print_dict(slide_meta)
print("tile_meta:")
print_dict(tile_meta)

# read the contents of this file in structured format and compare feature shapes
print(f"\nfeatures (flattened): \n\t{features.shape}, {features.dtype}")
features, _, _, _ = read_record(serialized, variables, structured=True)
print(f"features (structured): \n\t{features.shape}, {features.dtype}")

features (flattened): 
	(8926, 1280), <dtype: 'float32'>
labels:
	age: [36.], <dtype: 'float32'>
	dataset: [b'GBM'], <dtype: 'string'>
	event: [b'Deceased'], <dtype: 'string'>
	grade: [4.], <dtype: 'float32'>
	site: [b'27'], <dtype: 'string'>
	subject: [b'TCGA-27-1837'], <dtype: 'string'>
	time: [427], <dtype: 'int64'>
slide_meta:
	filename: [b'/data/ece495_2022/wsi/gbm/TCGA-27-1837-01Z-00-DX1.27b8fc01-d46e-459e-b4b9-6b80bf7dc6a9.svs'], <dtype: 'string'>
	level: [8], <dtype: 'int64'>
	read_magnification: [20.], <dtype: 'float32'>
	returned_magnification: [20.], <dtype: 'float32'>
	scan_magnification: [20.], <dtype: 'float32'>
	slide_name: [b'TCGA-27-1837-01Z-00-DX1.27b8fc01-d46e-459e-b4b9-6b80bf7dc6a9'], <dtype: 'string'>
	slide_group: [b'TCGA-27-1837-01Z-00-DX1.27b8fc01-d46e-459e-b4b9-6b80bf7dc6a9'], <dtype: 'string'>
	number_pixel_rows_for_chunk: [2048], <dtype: 'int64'>
	number_pixel_columns_for_chunk: [2048], <dtype: 'int64'>
	number_pixel_rows_for_tile: [256], <dtype: 'int64'>
	nu

### Dataset-generating functions and parameters
`train_ds` and `validation_ds` are tf.data.Dataset objects used by Keras/TensorFlow to read .tfr files. These objects optimize reading performance by prefetching and in-memory shuffling and caching of the data, memory permitting.

This cell also defines parameter values for training (`prefetch`, `cache`, `shuffle`, and `batch`). Parameters used for the feature extraction to create .tfr files are also defined here (`magnification`, `d`, `t`, `o`). Feature extraction parameters are helpful for understanding the location and extent of instances within the whole-slide image, and can be used for example to generate heatmaps.

In [5]:
# parameters - dataset
prefetch = 1 # Load the next file while processing the current file
cache = False # True caches data in memory after initial read (possible in Colab Pro)
shuffle = False # True shuffles the entire dataset in memory each epoch (possible in Colab Pro)
batch = 1 # this is required since each sample has different sizes

# parameters - feature extraction
magnification = 20 # objective lens magnification used for analysis
d = 1280 # dimensionality of features extracted at each patch
t = 256 # size of tiles in pixels at scale 'magnification'
o = 0 # overlap between adjacent tiles

# parameters - label time
tau = 365.25 * 2

# define a function to calculate the survival label from the time, event
def threshold(value, key, cond=lambda x: tf.cast(x, tf.float32)>=tau):
    return tf.one_hot(tf.cast(cond(value[key]), tf.int32), depth=2)

# define dataset generating function - read .tfr records, decode serialized 
# contents, and transform label to one-hot
def dataset(files, reads=4, structured=False, shuffle=False, cache=False):
  ds = tf.data.TFRecordDataset(files, num_parallel_reads=reads)
  ds = ds.map(lambda x: read_record(x, variables, structured=structured))
  if shuffle:
    ds = ds.shuffle(len(files))
  ds = ds.map(lambda x, y, z, _: (x, threshold(y, 'time')[0]))
  ds = ds.batch(batch).prefetch(prefetch)

  # cache if memory permits
  if cache:
    ds = ds.cache()

  return ds

## Baseline 1 - Average pooling model for flattened format

This model average pools over the instances and then predicts from a single 2-unit dense layer. Hinge loss is used for optimization.

This model can be trained efficiently on CPU (~90 seconds / epoch on CPU without caching).

In [6]:
# create a flattened dataset
train_flat = dataset(train, shuffle=shuffle, cache=cache)
validation_flat = dataset(validation, cache=cache)

# metrics to monitor performance during training
metrics = [tf.keras.metrics.BinaryAccuracy(),
           tf.keras.metrics.AUC(curve='ROC'),
           F1(), 
           Mcc()]

# build and compile model
input_layer = tf.keras.Input([None, d])
pooled = tf.reduce_mean(input_layer, axis=1)
transform = tf.keras.layers.Dense(2, activation='softmax', name='softmax')(pooled)
model = tf.keras.Model(inputs=input_layer, outputs=transform)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss={'softmax': tf.keras.losses.Hinge()},
              metrics={'softmax': metrics})

# train model
model.fit(train_flat, batch_size=batch, epochs=5, 
          validation_data=validation_flat, validation_freq=1)

# save model
model.save('./baseline1')

Epoch 1/5
417/417 [==============================] - 26s 48ms/step - loss: 0.9944 - binary_accuracy: 0.5276 - auc: 0.5049 - f1: 0.0000e+00 - mcc: nan - val_loss: 0.9725 - val_binary_accuracy: 0.5283 - val_auc: 0.5977 - val_f1: 0.0000e+00 - val_mcc: nan
Epoch 2/5
417/417 [==============================] - 10s 24ms/step - loss: 0.9523 - binary_accuracy: 0.5803 - auc: 0.6376 - f1: 0.2489 - mcc: 0.2051 - val_loss: 0.9288 - val_binary_accuracy: 0.6415 - val_auc: 0.6992 - val_f1: 0.4571 - val_mcc: 0.3172
Epoch 3/5
417/417 [==============================] - 9s 22ms/step - loss: 0.9121 - binary_accuracy: 0.6859 - auc: 0.7253 - f1: 0.5893 - mcc: 0.3840 - val_loss: 0.8933 - val_binary_accuracy: 0.7547 - val_auc: 0.7480 - val_f1: 0.6977 - val_mcc: 0.5195
Epoch 4/5
417/417 [==============================] - 10s 22ms/step - loss: 0.8851 - binary_accuracy: 0.7002 - auc: 0.7480 - f1: 0.6334 - mcc: 0.4038 - val_loss: 0.8698 - val_binary_accuracy: 0.7170 - val_auc: 0.7547 - val_f1: 0.6667 - val_mcc: 0.

## Baseline 2 - A simple convolutional model for structured format

Convolve over the height and width dimensions of the height x width x d structured tensor. Apply a single dense layer to make a prediction. Height and width dimensions are variable.

This model benefits significantly from GPU acceleration (~30 minutes / epoch on CPU without caching).

Caching is not recommended for this model, since the conversion from flattened to structured format significantly inflates the data.

In [ ]:
# create a structured dataset
train_structured = dataset(train, structured=True, shuffle=shuffle, cache=cache)
validation_structured = dataset(validation, structured=True, cache=cache)

# metrics to monitor performance during training
metrics = [tf.keras.metrics.BinaryAccuracy(),
           tf.keras.metrics.AUC(curve='ROC'),
           F1(), 
           Mcc()]

# build model
input_layer = tf.keras.Input([None, None, d])
convolved = tf.keras.layers.Conv2D(128, 3)(input_layer)
pooled = tf.keras.layers.GlobalMaxPooling2D()(convolved)
transform = tf.keras.layers.Dense(2, activation='softmax', name='softmax')(pooled)
model = tf.keras.Model(inputs=input_layer, outputs=transform)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss={'softmax': tf.keras.losses.BinaryCrossentropy()},
              metrics={'softmax': metrics})

# train model
model.fit(train_structured, batch_size=batch, epochs=5, 
          validation_data=validation_structured, validation_freq=1)

# save model
model.save('./baseline2')

Epoch 1/5
417/417 [==============================] - 69s 148ms/step - loss: 0.7237 - binary_accuracy: 0.6355 - auc_1: 0.6518 - f1: 0.6000 - mcc: 0.2666 - val_loss: 0.5662 - val_binary_accuracy: 0.7547 - val_auc_1: 0.7825 - val_f1: 0.7636 - val_mcc: 0.5223
Epoch 2/5
149/417 [=========>....................] - ETA: 18s - loss: 0.4678 - binary_accuracy: 0.8322 - auc_1: 0.8677 - f1: 0.8201 - mcc: 0.6630

## Baseline 3 - Attention model for flattened format

This model is similar to CLAM and uses attention to perform a weighted average pooling of instances. As in CLAM, instance features pass through a single dense layer prior to pooling. The model outputs predictions and also attention weights that can be used for visualizing salient instances.

This model benefits slightly from GPU acceleration (~5 min / epoch on CPU without caching).

In [ ]:
# create a flattened dataset
train_flat = dataset(train, shuffle=shuffle, cache=cache)
validation_flat = dataset(validation, cache=cache)

# create a list of metrics to monitor performance during training
metrics = [tf.keras.metrics.BinaryAccuracy(),
           tf.keras.metrics.AUC(curve='ROC'),
           F1(), 
           Mcc()]

# build and compile model
model = attention_flat(d)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss={'softmax': tf.keras.losses.Hinge(), "attention_weights": None},
              metrics={'softmax': metrics, "attention_weights": None})

# train model
model.fit(train_flat, batch_size=batch, epochs=5, 
          validation_data=validation_flat, validation_freq=1)

# save model
model.save('./baseline3')